## TV Rating Analysis on Korea


#### First step : Collect Data from Nielsen Korea

Nielsen Korea publish the top 20 TV programs by ratings on Korea every day.  
The data is updated every day. (www.nielsenkorea.co.kr)

We can collect the data from Nielsen Korea website with requests and BeautifulSoup.

In [ ]:
# import libraries
from bs4 import BeautifulSoup
import requests


- Nielsen's web page shows data by calling url ("http://www.nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1") 
- with sub_menu(tv category), area, and date as parameters.
- We can get the data by using requests and BeautifulSoup.

In [3]:
# set URL and parameters for sample data
url = f"http://www.nielsenkorea.co.kr/tv_terrestrial_day.asp?menu=Tit_1"
url += f"&sub_menu=1_1"  # terrestrial TV
url += f"&area=01" # GSE(Greater Seoul Area)
url += f"&begin_date=20260207" # Date

In [5]:
# get page and parse html with BS4
response = requests.get(url)        
parsed_html = BeautifulSoup(response.content.decode(
    'utf-8', 'replace'), 'html.parser')



In [6]:
parsed_html

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="css/style.css" rel="StyleSheet" type="text/css"/>
<link href="images/favicon.ico" rel="icon"/>
<title>Nielsen Korea</title>
</head>
<body onload="showhide('1', '3');Load_Sub_Menu('1_1');">
<script language="JavaScript">
    function showhide(num, p_totcount) {

        if (num == undefined) return;

        // for (i=1; i<=main_cnt; i++)   { 
        for (i = 1; i <= p_totcount; i++) {

            menu = eval("document.all.block" + i + ".style");

            if (num == i) {
                if (menu.display == "block") {
                    menu.display = "none";
                }
                else {
                    menu.display = "block";
                }

            } e